<font size=6  color= 'blue' > <b>[LEPL1507] - PROJET P4</b> <br>
<br><br>
Airson Alexis<br>
Beniffou Ibrahim <br>
Henneaux Lucas <br>
Lemaire Antoine <br>
Smith Marielle <br>
Canon Théo<br>
<div style="text-align: right"> </div>

<br><br>
</font>


In [10]:
import keras 
import cv2
from scipy.misc import imread

In [14]:
#preprocessing of ppm dataset 
path = 'BelgiumTSC_Training\Training'
img = cv2.imread('BelgiumTSC_Testing\Testing\00000')
